<h1>IBM Capstone Project</h1>

<h2> Week 5 Final Project </h2>
<h3><i>Opening a New Shopping Mall in Delhi, India</i></h3>

* Build a dataframe of neighbourshoods in Delhi,India by webscrapping from Wikipedia page.
* Get coordinates of the neighbourhoods.
* Using Foursquare API to obtain Shopping Mall Data.
* Cluster the neighbourhoods.
* Selection of best cluster to open a new shoping mall.


<h2>Installing and Importing Required Libraries

In [2]:
!pip install geocoder

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<h2> Scrapping data from Wikipedia Page Using BeautifulSoup

In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Delhi").text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,List of neighbourhoods of Delhi
1,Ashok Nagar (Delhi)
2,Ashok Vihar
3,Ashram Chowk
4,Babarpur


In [9]:
kl_df = kl_df.iloc[1:,]

In [10]:
# print the number of rows of the dataframe
kl_df.head()

,Neighborhood
1,Ashok Nagar (Delhi)
2,Ashok Vihar
3,Ashram Chowk
4,Babarpur
5,"Badarpur, Delhi"


<h2> Getting coordinates Of Neighbourhoods

In [11]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [13]:
coords

[[28.692230000000052, 77.30124000000006],
 [28.690390000000036, 77.17607000000004],
 [28.710598435255907, 77.32696519316737],
 [28.68219848827012, 77.28533883577524],
 [28.50738000000007, 77.30346000000003],
 [28.652493114632577, 77.1294978104143],
 [28.800590000000057, 77.03473000000008],
 [28.549540000000036, 77.18167000000005],
 [28.699880000000064, 77.25906000000003],
 [28.595060000000046, 77.18573000000004],
 [28.656270000000063, 77.23232000000007],
 [28.67661000000004, 77.21763000000004],
 [28.633940000000052, 77.21968000000004],
 [28.60761000000008, 77.08714000000003],
 [28.654597885415757, 77.2333966005242],
 [28.62832000000003, 77.24727000000007],
 [28.60486000000003, 77.08511000000004],
 [28.71768000000003, 77.26517000000007],
 [28.560590000000047, 77.24678000000006],
 [28.57298000000003, 77.23357000000004],
 [28.600000000000023, 77.13333000000006],
 [28.699110000000076, 77.19105000000008],
 [28.59196002995221, 77.1601250847616],
 [28.684700000000078, 77.32774000000006],
 [28

In [14]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [16]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(153, 3)


,Neighborhood,Latitude,Longitude
1,Ashok Nagar (Delhi),28.690390,77.176070
2,Ashok Vihar,28.710598,77.326965
3,Ashram Chowk,28.682198,77.285339
4,Babarpur,28.507380,77.303460
5,"Badarpur, Delhi",28.652493,77.129498
6,Bali Nagar,28.800590,77.034730
7,Bawana,28.549540,77.181670
8,Ber Sarai,28.699880,77.259060
9,Bhajanpura,28.595060,77.185730
10,Chanakyapuri,28.656270,77.232320


<h2> Replacing Nan values Manually  

In [17]:
kl_df['Latitude'] = kl_df['Latitude'].replace(np.nan, 28.700939)
kl_df['Longitude'] = kl_df['Longitude'].replace(np.nan, 77.272102)

In [18]:
kl_df

,Neighborhood,Latitude,Longitude
1,Ashok Nagar (Delhi),28.690390,77.176070
2,Ashok Vihar,28.710598,77.326965
3,Ashram Chowk,28.682198,77.285339
4,Babarpur,28.507380,77.303460
5,"Badarpur, Delhi",28.652493,77.129498
6,Bali Nagar,28.800590,77.034730
7,Bawana,28.549540,77.181670
8,Ber Sarai,28.699880,77.259060
9,Bhajanpura,28.595060,77.185730
10,Chanakyapuri,28.656270,77.232320


In [19]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

<h2>Creating a map of Delhi with Neighbourhoods on Top

In [20]:
# get the coordinates of Delhi
address = 'Delhi, India'

geolocator = Nominatim(user_agent="locdata")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi, India 28.6517178, 77.2219388.


In [21]:
# create map of Delhi using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [22]:
# save the map as HTML file
map_kl.save('map_kl.html')

<h2>Using the Foursquare API to explore the Neighbourhoods

In [23]:
# define Foursquare Credentials and Version
CLIENT_ID = 'Y4UCT3H1ZLE4V5R3BOHQ0NA0HJTW5XW5BOH22Z5SGVBN4VK2' # your Foursquare ID
CLIENT_SECRET = 'ENCCBE4YIKDCI1IFYRRIPIUEWFHQW4V5Q514FMICKZ34EINI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4UCT3H1ZLE4V5R3BOHQ0NA0HJTW5XW5BOH22Z5SGVBN4VK2
CLIENT_SECRET:ENCCBE4YIKDCI1IFYRRIPIUEWFHQW4V5Q514FMICKZ34EINI


<h2>Getting top 100 venues within a radius of 2000 meters

In [36]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?Y4UCT3H1ZLE4V5R3BOHQ0NA0HJTW5XW5BOH22Z5SGVBN4VK2={}&ENCCBE4YIKDCI1IFYRRIPIUEWFHQW4V5Q514FMICKZ34EINI={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [37]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6325, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Ashok Nagar (Delhi),28.69039,77.17607,Major Dhyan Chand Sports Complex,28.684029,77.167487,Athletics & Sports
1,Ashok Nagar (Delhi),28.69039,77.17607,Bellagio,28.696361,77.180021,Asian Restaurant
2,Ashok Nagar (Delhi),28.69039,77.17607,Subway,28.696321,77.179983,Sandwich Place
3,Ashok Nagar (Delhi),28.69039,77.17607,Rahul Egg Corner,28.688240,77.168599,Snack Place
4,Ashok Nagar (Delhi),28.69039,77.17607,Subway.,28.695571,77.171964,Sandwich Place


<h2> Checking the returned venues from each neighbourhood

In [38]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Ashok Nagar (Delhi),24,24,24,24,24,24
Ashok Vihar,4,4,4,4,4,4
Ashram Chowk,6,6,6,6,6,6
Babarpur,5,5,5,5,5,5
"Badarpur, Delhi",53,53,53,53,53,53
Bali Nagar,1,1,1,1,1,1
Bawana,98,98,98,98,98,98
Ber Sarai,8,8,8,8,8,8
Bhajanpura,75,75,75,75,75,75


<h2>Finding number of Unique Venues

In [39]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 218 uniques categories.


In [40]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Athletics & Sports', 'Asian Restaurant', 'Sandwich Place',
       'Snack Place', 'Indian Restaurant', 'Pizza Place',
       'South Indian Restaurant', 'Department Store',
       'Fast Food Restaurant', 'Coffee Shop', 'Market', 'Dessert Shop',
       'Basketball Court', 'Train Station', 'Light Rail Station',
       'Restaurant', 'Vegetarian / Vegan Restaurant', 'ATM', 'Print Shop',
       'Child Care Service', 'Park', 'Indian Sweet Shop', 'Café',
       'American Restaurant', 'Donut Shop', 'Bakery', 'Diner',
       'Hookah Bar', 'BBQ Joint', 'Garden Center', 'Sports Bar', 'Hotel',
       'Pub', 'Multiplex', 'Fried Chicken Joint',
       'Furniture / Home Store', 'Shopping Mall', 'Bar',
       'Chinese Restaurant', 'Clothing Store', 'Flea Market',
       'Gourmet Shop', 'Garden', 'Cafeteria', 'Ice Cream Shop', 'Gym',
       'Gym / Fitness Center', 'Playground', 'Art Gallery',
       'Mediterranean Restaurant'], dtype=object)

In [41]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

<h2> One hot encoding Neighbourhoods

In [42]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6325, 219)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Building,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Cafeteria,Café,Child Care Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ashok Nagar (Delhi),0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

<h2> Using mean to find frequency of occurence in each category

In [43]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(152, 219)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Building,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Cafeteria,Café,Child Care Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Ashok Nagar (Delhi),0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

<h2>Grouping and Creating New DataFrame for Shoping Mall Data Only

In [44]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

46

In [45]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [46]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Ashok Nagar (Delhi),0.000000
1,Ashok Vihar,0.000000
2,Ashram Chowk,0.000000
3,Babarpur,0.000000
4,"Badarpur, Delhi",0.037736


<h2>Clusturing Neighbourhoods

In [47]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

In [48]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [49]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Ashok Nagar (Delhi),0.000000,0
1,Ashok Vihar,0.000000,0
2,Ashram Chowk,0.000000,0
3,Babarpur,0.000000,0
4,"Badarpur, Delhi",0.037736,2


In [50]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(152, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,0,28.690390,77.176070
1,Ashok Vihar,0.000000,0,28.710598,77.326965
2,Ashram Chowk,0.000000,0,28.682198,77.285339
3,Babarpur,0.000000,0,28.507380,77.303460
4,"Badarpur, Delhi",0.037736,2,28.652493,77.129498


In [51]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(152, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,0,28.690390,77.176070
100,Pandav Nagar,0.000000,0,28.669330,77.091730
98,Palika Bazaar,0.000000,0,28.546510,77.245460
97,"Palam, Delhi",0.000000,0,28.631560,77.219590
96,Old Delhi,0.000000,0,28.591040,77.091180
95,Okhla,0.000000,0,28.654340,77.232580
94,Nizamuddin West,0.000000,0,28.532470,77.278390
93,Nizamuddin East,0.000000,0,28.587380,77.246310
92,Nigambodh Ghat,0.000000,0,28.601226,77.264530
91,New Moti Bagh,0.000000,0,28.664710,77.236330


<h2>Using Follium to Visualize Clusters

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h2>Examining Clusters for Observation

<h4><i>Cluster 0

In [54]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ashok Nagar (Delhi),0.000000,0,28.690390,77.176070
100,Pandav Nagar,0.000000,0,28.669330,77.091730
98,Palika Bazaar,0.000000,0,28.546510,77.245460
97,"Palam, Delhi",0.000000,0,28.631560,77.219590
96,Old Delhi,0.000000,0,28.591040,77.091180
95,Okhla,0.000000,0,28.654340,77.232580
94,Nizamuddin West,0.000000,0,28.532470,77.278390
93,Nizamuddin East,0.000000,0,28.587380,77.246310
92,Nigambodh Ghat,0.000000,0,28.601226,77.264530
91,New Moti Bagh,0.000000,0,28.664710,77.236330


<h4><i>Cluster 1

In [55]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
62,Lajpat Nagar,0.100000,1,28.63875,77.27592
19,Defence Colony,0.111111,1,28.60000,77.13333
132,"Shanti Niketan, Delhi",0.111111,1,28.64752,77.27082
113,Roop Nagar,0.125000,1,28.59051,77.12026
103,Pitam Pura,0.107143,1,28.63903,77.29597
49,"Kamla Nagar, New Delhi",0.117647,1,28.52171,77.08644
111,"Rani Bagh, Delhi",0.111111,1,28.73356,77.10401


<h4><i>Cluster 2

In [56]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
147,Vikram Vihar,0.031250,2,28.646420,77.306150
135,Shiv Ram Park,0.038462,2,28.652640,77.121430
149,West Patel Nagar,0.038462,2,28.582890,77.300728
142,Urdu Bazaar,0.050847,2,28.531510,77.150200
4,"Badarpur, Delhi",0.037736,2,28.652493,77.129498
139,Tikri Kalan,0.041667,2,28.640890,77.089220
63,Laxmi Nagar (Delhi),0.049180,2,28.578150,77.206180
120,Sarita Vihar,0.042857,2,28.575600,77.193640
69,Majnu-ka-tilla,0.030000,2,28.533850,77.206940
71,Mayapuri,0.040000,2,28.607718,77.290668


<b>Observations:</b>
<br>
Most of the shopping malls are concentrated in the outer area of Delhi city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.